## Final Project Submission

Please fill out:
* Student name: John Harrigan
* Student pace: full time
* Scheduled project review date/time: 10/7/2022
* Instructor name: Joseph Mata
* Blog post URL:


In [389]:
import pandas as pd
import sqlite3
import re
import seaborn as sns

In [449]:
bom_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
ret_info_df = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep = '\t')
ret_reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep = '\t', encoding='unicode_escape')
tmdb_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz', index_col=0)
tn_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
conn = sqlite3.Connection('./zippedData/john/im.db')

## cleaning
here we start the code

In [450]:
pd.set_option('display.float_format', lambda x: '%.f' % x)
tmdb_df.drop('original_title', axis=1, inplace = True)
tmdb_df['genre_ids'].replace(to_replace= '[]', value = '[7]', inplace = True)
getGenre = {
    '7': 'undecided', '12': 'Adventure', '14': 'Fantasy', '16': 'Animation', '18': 'Drama', '27': 'Horror', '28': 'Action',
    '35': 'Comedy', '36': 'History', '37': 'Western', '53': 'Thriller', '80': 'Crime', '99': 'Documentary', 
    '878': 'Science Fiction','9648': 'Mystery','10402': 'Music','10749':'Romance','10751': 'Family','10752': 'War','10770': 'TV'
}
def convert(glist):
    description = ''
    for num in glist.strip('][').split(', '):
        description += getGenre[num]
        description += ', '
    return description.rstrip(', ')
tmdb_df['genre_ids'] = tmdb_df['genre_ids'].map(lambda x: convert(x))
tmdb_df.drop(['id', 'original_language'], axis=1, inplace=True)
tmdb_df = tmdb_df[['title', 'release_date', 'genre_ids', 'popularity', 'vote_average', 'vote_count']]
tmdb_df.drop_duplicates(inplace=True)

In [451]:
tn_df['release_date'] = pd.to_datetime(tn_df['release_date'])
tn_df['production_budget'] = tn_df['production_budget'].map(lambda x: int(x.replace('$','').replace(',','')))
tn_df['domestic_gross'] = tn_df['domestic_gross'].map(lambda x: int(x.replace('$','').replace(',','')))
tn_df['worldwide_gross'] = tn_df['worldwide_gross'].map(lambda x: int(x.replace('$','').replace(',','')))

In [452]:
bom_df['foreign_gross'] = bom_df['foreign_gross'].fillna('0')
bom_df['foreign_gross'] = bom_df['foreign_gross'].map(lambda x: x.replace(',', '')).map(lambda x: float(x))
bom_df['domestic_gross'] = bom_df['domestic_gross'].astype(float)

bomtn = pd.merge(bom_df, tn_df, how='outer', left_on='title', right_on='movie')

bomtn['title'] = bomtn['title'].fillna(bomtn['movie'])
bomtn['domestic_gross_x'] = bomtn['domestic_gross_x'].fillna(bomtn['domestic_gross_y'])
bomtn['foreign_gross'] = bomtn['foreign_gross'].fillna(bomtn['worldwide_gross'] - bomtn['domestic_gross_y'])
bomtn['release_date'] = bomtn['release_date'].fillna(pd.to_datetime(bomtn['year'], format='%Y'))
bomtn.drop(bomtn.columns[[1, 4, 5, 7, 9, 10]], axis=1, inplace=True)
bomtn = bomtn[['title','release_date','production_budget', 'domestic_gross_x', 'foreign_gross']]
bomtn.rename(columns={'domestic_gross_x':'domestic_gross'}, inplace=True)
bomtn['worldwide_gross'] = bomtn['domestic_gross'] + bomtn['foreign_gross']
bomtn.dropna(inplace=True)
bomtn = bomtn[bomtn['worldwide_gross'] != 0.0]

In [302]:
mvi = pd.read_sql('''
SELECT movie_id, (primary_title) AS title, (start_year) year, (runtime_minutes) AS runtime, genres,
(averagerating) AS avg_rating, numvotes
FROM movie_basics
LEFT JOIN movie_ratings
USING(movie_id)

''', conn)

directors = pd.read_sql('''
SELECT movie_id, (primary_name) AS director
FROM principals
JOIN persons
USING(person_id)
WHERE category = 'director'
''', conn)
directors = directors.drop_duplicates()
directors = directors.groupby('movie_id').agg(lambda x: x.tolist()).reset_index()

actors = pd.read_sql('''
SELECT movie_id, (primary_name) AS actors
FROM principals
JOIN persons
USING(person_id)
WHERE category = 'actor'
ORDER BY movie_id
''', conn)
actors = actors.drop_duplicates()
actors = actors.groupby('movie_id').agg(lambda x: x.tolist()).reset_index()

draft1 = pd.merge(mvi,directors, how = 'left', on = 'movie_id')
sql_df = pd.merge(draft1, actors, how = 'left', on = 'movie_id')
sql_df['runtime'].fillna(sql_df['runtime'].median(), inplace = True)
sql_df.drop('movie_id', axis=1, inplace=True)

In [318]:
sql_df.head(1)

,title,year,runtime,genres,avg_rating,numvotes,director,actors
0,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,[Harnam Singh Rawail],"[Dilip Kumar, Balraj Sahni, Sanjeev Kumar]"


In [308]:
tmdb_df.head(1)

,title,release_date,genre_ids,popularity,vote_average,vote_count
0,Harry Potter and the Deathly Hallows: Part 1,2010-11-19,"Adventure, Fantasy, Family",33.533,7.7,10788


In [347]:
tmdb_df['popularity'].corr(tmdb_df['vote_count'])

0.68528527625959

In [323]:
tmdbsql.head(1)

,title,year,runtime,genres,avg_rating,numvotes,director,actors,release_date,genre_ids,popularity,vote_average,vote_count
0,Sunghursh,2013.0,175.0,"Action,Crime,Drama",7.0,77.0,[Harnam Singh Rawail],"[Dilip Kumar, Balraj Sahni, Sanjeev Kumar]",NaN,NaN,NaN,NaN,NaN


In [343]:
tmdbsql = pd.merge(sql_df, tmdb_df,how ='outer', on='title' )
tmdbsql['avg_rating'] = tmdbsql['avg_rating'].fillna(tmdbsql['vote_average'])
tmdbsql['numvotes'] = tmdbsql['numvotes'].fillna(tmdbsql['vote_count'])
tmdbsql['genres'] = tmdbsql['genres'].fillna(tmdbsql['genre_ids'])

In [344]:
tmdbsql.isna().sum()

title                0
year              9771
runtime           9771
genres            5154
avg_rating       69660
numvotes         69660
director         27752
actors           61292
release_date    126595
genre_ids       126595
popularity      126595
vote_average    126595
vote_count      126595
dtype: int64